In [1]:
%pip install openpyxl

--- Logging error ---
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pip/_internal/utils/logging.py", line 177, in emit
    self.console.print(renderable, overflow="ignore", crop=False, style=style)
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pip/_vendor/rich/console.py", line 1673, in print
    extend(render(renderable, render_options))
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pip/_vendor/rich/console.py", line 1305, in render
    for render_output in iter_render:
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pip/_internal/utils/logging.py", line 134, in __rich_console__
    for line in lines:
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pip/_vendor/rich/segment.py", line 249, in split_lines
    for segment in segments:
  File "/Library/

In [2]:
import pandas as pd
import openpyxl

In [41]:
# Read in campaign finance csv
df = pd.read_csv('rochagarcia_cf.csv')
df.head()

,Report Id:,Name:,Contact Type:,Report Type:,Amount:,strVal,Cand/Committee:,Election Date:,Id,ReportId,FilerName,Report Type Code:,Count:,TransDate:,CreatedDt:
0,View Report,"McBride, Elissa",Treasurer of Committee,8th day before June 2019 Run-off,0.0,Notice From Political Committees,Adriana Garcia,6/8/19,65.0,2339.0,Adriana Garcia,C,1.0,5/30/19 23:43,5/28/19 23:03
1,NaN,1625 L Street NW,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,"Washington, DC 20036",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,View Report,TEXAS REALTORS PAC,Expenditure,8th day before June 2019 Run-off,0.0,Notice From Political Committees,Adriana Garcia,6/8/19,66.0,2339.0,Adriana Garcia,C,2.0,5/30/19 23:45,5/28/19 23:03
4,NaN,PO Box 295305,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [42]:
# Read in campaign finance excel
excel_file_path = '/Users/jackturek/Desktop/Data Data Data/Personal Projects/Public Policy Stuff/Voting, Political Science & Elections/SATX Politics/Campaign Finance/rochagarcia_cf.xlsx'

In [43]:
wb = openpyxl.load_workbook(excel_file_path)
sheet = wb.active

In [44]:
# Extract hyperlinks from Report Id: column (column A)
hyperlinks = []
for row in sheet.iter_rows(min_row=1, max_col=1, values_only=False):
    cell = row[0]
    if cell.hyperlink:
        hyperlinks.append(cell.hyperlink.target)
    else:
        hyperlinks.append(None)

In [45]:
# Filter out rows where Report Id: is NaN
clean_df = df.dropna(subset=['Report Id:']).copy()

In [46]:
# Create correct hyperlink list for cleaned df
clean_hyperlinks = []
link_idx = 0

for i in range(len(clean_df)):
    while link_idx < len(hyperlinks) and hyperlinks[link_idx] is None:
        link_idx += 1   # Skip empty or None hyperlinks
    if link_idx < len(hyperlinks):
        clean_hyperlinks.append(hyperlinks[link_idx])
        link_idx += 1
    else:
        clean_hyperlinks.append(None)

In [47]:
# Add clean hyperlinks as new ReportLink column
clean_df['ReportLink'] = clean_hyperlinks

In [48]:
clean_df.head()

,Report Id:,Name:,Contact Type:,Report Type:,Amount:,strVal,Cand/Committee:,Election Date:,Id,ReportId,FilerName,Report Type Code:,Count:,TransDate:,CreatedDt:,ReportLink
0,View Report,"McBride, Elissa",Treasurer of Committee,8th day before June 2019 Run-off,0.0,Notice From Political Committees,Adriana Garcia,6/8/19,65.0,2339.0,Adriana Garcia,C,1.0,5/30/19 23:43,5/28/19 23:03,https://webapp5.sanantonio.gov/CampFinHistoric...
3,View Report,TEXAS REALTORS PAC,Expenditure,8th day before June 2019 Run-off,0.0,Notice From Political Committees,Adriana Garcia,6/8/19,66.0,2339.0,Adriana Garcia,C,2.0,5/30/19 23:45,5/28/19 23:03,https://webapp5.sanantonio.gov/CampFinHistoric...
6,View Report,"Mitchell, Stanley",Treasurer of Committee,July 15: Semi-Annual 2019,0.0,Notice From Political Committees,Adriana Garcia,6/8/19,67.0,2359.0,Adriana Garcia,C,3.0,7/15/19 09:59,7/14/19 09:40,https://webapp5.sanantonio.gov/CampFinHistoric...
9,View Report,TEXAS REALTORS PAC,Expenditure,8th Day Before General Election 2021,0.0,Notice From Political Committees,Adriana Garcia,5/1/21,72.0,2639.0,Adriana Garcia,C,4.0,4/20/21 19:39,4/20/21 19:34,https://webapp5.sanantonio.gov/CampFinHistoric...
12,View Report,"McBride, Elissa",Treasurer of Committee,July 15: Semi-Annual,0.0,Notice From Political Committees,Adriana Garcia,NaN,75.0,2760.0,Adriana Garcia,C,5.0,7/13/21 09:36,7/12/21 11:26,https://webapp5.sanantonio.gov/CampFinHistoric...


In [49]:
clean_df.to_csv('rochagarcia_cf_clean.csv', index=False)